In [1]:
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification, TrainingArguments, Trainer

from datasets import load_dataset,concatenate_datasets, load_metric
import pandas as pd
import numpy as np

None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [2]:
wnut = load_dataset("wnut_17")
wnut

Generating train split:   0%|          | 0/3394 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1009 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1287 [00:00<?, ? examples/s]

Dataset wnut_17 downloaded and prepared to C:/Users/Serdar Arslan/.cache/huggingface/datasets/wnut_17/wnut_17/1.0.0/077c7f08b8dbc800692e8c9186cdf3606d5849ab0e7be662e6135bb10eba54f9. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 3394
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1009
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags'],
        num_rows: 1287
    })
})

In [3]:
label_list = wnut["train"].features[f"ner_tags"].feature.names
id2tag = {id: tag for id, tag in enumerate(label_list)}
id2tag

{0: 'O',
 1: 'B-corporation',
 2: 'I-corporation',
 3: 'B-creative-work',
 4: 'I-creative-work',
 5: 'B-group',
 6: 'I-group',
 7: 'B-location',
 8: 'I-location',
 9: 'B-person',
 10: 'I-person',
 11: 'B-product',
 12: 'I-product'}

In [4]:
# merge train & validation sets
from datasets import concatenate_datasets

train_dataset = concatenate_datasets([wnut["train"],wnut["validation"]])
train_dataset

Dataset({
    features: ['id', 'tokens', 'ner_tags'],
    num_rows: 4403
})

In [5]:
ith_example=2

print(wnut["train"][ith_example]['tokens'])
print([id2tag[label] for label in train_dataset[ith_example]['ner_tags']])

['Pxleyes', 'Top', '50', 'Photography', 'Contest', 'Pictures', 'of', 'August', '2010', '...', 'http://bit.ly/bgCyZ0', '#photography']
['B-corporation', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O']


In [6]:
model_name = "bert-base-uncased"

tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [7]:
#buraya bizim veri setlerini tokenize edip eklememiz lazım
token_str = list(map(str.lower, ['YETI', 'RESCUE', 'AD000092', ',', 'SIZE50', '(', 'PARTS', 'FOR', 'PARAGLIDER', ')']))

print(token_str)
tokenized_input = tokenizer(token_str, is_split_into_words=True)
print(tokenized_input)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
print("Tokens:----------------------------")
print(tokens)

['yeti', 'rescue', 'ad000092', ',', 'size50', '(', 'parts', 'for', 'paraglider', ')']
{'input_ids': [101, 2664, 2072, 5343, 4748, 8889, 8889, 2683, 2475, 1010, 2946, 12376, 1006, 3033, 2005, 11498, 25394, 4063, 1007, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Tokens:----------------------------
['[CLS]', 'yet', '##i', 'rescue', 'ad', '##00', '##00', '##9', '##2', ',', 'size', '##50', '(', 'parts', 'for', 'para', '##gli', '##der', ')', '[SEP]']


In [8]:
tokenized_input = tokenizer(wnut["train"][2]["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
#tokenized
print(tokens)

['[CLS]', 'p', '##xley', '##es', 'top', '50', 'photography', 'contest', 'pictures', 'of', 'august', '2010', '.', '.', '.', 'http', ':', '/', '/', 'bit', '.', 'l', '##y', '/', 'b', '##gc', '##y', '##z', '##0', '#', 'photography', '[SEP]']


In [9]:
#bizim data
df = pd.read_excel('data/test.xlsx', sheet_name='sheet1')
df.head(10)

,id,product,tokens,ner_tags
0,0,(AIR COMBAT SIMULATOR PARTS) DIGITAL OUTPUT 32...,"['(', 'AIR', 'COMBAT', 'SIMULATOR', 'PARTS', '...","['O ', 'B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', ..."
1,1,(AIR COMBAT SIMULATOR PARTS)AMPLIFIER-4-CHANNE...,"['(', 'AIR', 'COMBAT', 'SIMULATOR', 'PARTS', '...","['O ', 'B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', ..."
2,2,(AIR COMBAT SIMULATOR PARTS)AMPLIFIER-4-CHANNE...,"['(', 'AIR', 'COMBAT', 'SIMULATOR', 'PARTS', '...","['O ', 'B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', ..."
3,3,(AIR COMBAT SIMULATOR PARTS)AMPLIFIER-4-CHANNE...,"['(', 'AIR', 'COMBAT', 'SIMULATOR', 'PARTS', '...","['O ', 'B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', ..."
4,4,(AIR COMBAT SIMULATOR PARTS)AMPLIFIER-4-CHANNE...,"['(', 'AIR', 'COMBAT', 'SIMULATOR', 'PARTS', '...","['O ', 'B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', ..."
5,5,(AIR COMBAT SIMULATOR PARTS)TOGGLE SWITCH PNO-...,"['(', 'AIR', 'COMBAT', 'SIMULATOR', 'PARTS', '...","['O ', 'B_PRODUCT', 'I_PRODUCT', 'I_PRODUCT', ..."
6,6,(FOC SHIPMENT)LEADING EDGE GUARD (697039655) (...,"['(', 'FOC', 'SHIPMENT', ')', 'LEADING', 'EDGE...","['O ', 'O ', 'O ', 'O ', 'B_PRODUCT', 'I_PRODU..."
7,7,(FOC) (P/N:233A5512-3) SEAT GUARD ASSY,"['(', 'FOC', ')', '(', 'P', '/', 'N', ':', '23...","['O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ..."
8,8,"(FOC) (P/N:53727-22) ROD ASSY, FOLDED","['(', 'FOC', ')', '(', 'P', '/', 'N', ':', '53...","['O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ..."
9,9,(FOC) (P/N:53727-25) STRUT AY,"['(', 'FOC', ')', '(', 'P', '/', 'N', ':', '53...","['O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ', 'O ..."


Using custom data configuration default-c5fe293e99e203b5
Reusing dataset csv (C:\Users\pc\.cache\huggingface\datasets\csv\default-c5fe293e99e203b5\0.0.0\652c3096f041ee27b04d2232d41f10547a8fecda3e284a79a0ec4053c916ef7a)
100%|██████████| 1/1 [00:00<00:00, 1034.35it/s]


In [10]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [11]:
ds = load_dataset("csv", data_files="data/test2.csv")
ds

Using custom data configuration default-a645cf76485824ff


Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/Serdar Arslan/.cache/huggingface/datasets/csv/default-a645cf76485824ff/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags'],
        num_rows: 3243
    })
})

In [12]:
tokenized_df = ds.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/4 [00:00<?, ?ba/s]

IndexError: list index out of range

In [13]:
tokenized_wnut = wnut.map(tokenize_and_align_labels, batched=True)
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)

100%|██████████| 5/5 [00:00<00:00, 13.61ba/s]


In [14]:
tokenized_train_dataset[0]


{'id': '0',
 'tokens': ['@paulwalk',
  'It',
  "'s",
  'the',
  'view',
  'from',
  'where',
  'I',
  "'m",
  'living',
  'for',
  'two',
  'weeks',
  '.',
  'Empire',
  'State',
  'Building',
  '=',
  'ESB',
  '.',
  'Pretty',
  'bad',
  'storm',
  'here',
  'last',
  'evening',
  '.'],
 'ner_tags': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  7,
  8,
  8,
  0,
  7,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'input_ids': [101,
  1030,
  2703,
  17122,
  2009,
  1005,
  1055,
  1996,
  3193,
  2013,
  2073,
  1045,
  1005,
  1049,
  2542,
  2005,
  2048,
  3134,
  1012,
  3400,
  2110,
  2311,
  1027,
  9686,
  2497,
  1012,
  3492,
  2919,
  4040,
  2182,
  2197,
  3944,
  1012,
  102],
 'token_type_ids': [0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
 

In [16]:
id2tag[-100]='ignore'
exml=tokenized_train_dataset[2]

pd.DataFrame({'tokens':tokenizer.convert_ids_to_tokens(exml["input_ids"]), 'ner_labels':exml['labels'], 'ner_tags': [id2tag[label] for label in exml['labels']] })

,tokens,ner_labels,ner_tags
0,[CLS],-100,ignore
1,p,1,B-corporation
2,##xley,-100,ignore
3,##es,-100,ignore
4,top,0,O
5,50,0,O
6,photography,0,O
7,contest,0,O
8,pictures,0,O
9,of,0,O


In [17]:
#base model
from sklearn.dummy import DummyClassifier

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(pd.Series(tokenized_train_dataset['input_ids']).explode(), pd.Series(tokenized_train_dataset['labels']).explode().astype(str))
dummy_clf.score(pd.Series(tokenized_train_dataset['input_ids']).explode(), pd.Series(tokenized_train_dataset['labels']).explode().astype(str))

0.5888494815191806

In [18]:
exploded_values=pd.Series(tokenized_train_dataset['labels']).explode()
exploded_values=pd.DataFrame(exploded_values,columns=['B'])

most_frequent_elem_by_doc=pd.Series(tokenized_train_dataset['labels']).apply(lambda x:  max(set(x), key=x.count))
most_frequent_elem_by_doc=pd.DataFrame(most_frequent_elem_by_doc,columns=list('A'))

df_most_freq_token=exploded_values.merge(most_frequent_elem_by_doc, how='right', left_index=True, right_index=True)

dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(pd.Series(tokenized_train_dataset['input_ids']).explode(), df_most_freq_token['A'])
dummy_clf.score(pd.Series(tokenized_train_dataset['input_ids']).explode(), df_most_freq_token['A'])

0.7197897448947134

In [19]:
#model
#Data Collator

from transformers import DataCollatorForTokenClassification
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

model = AutoModelForTokenClassification.from_pretrained(model_name, num_labels=len(label_list))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

In [21]:
!pip install seqeval

  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16171 sha256=8dd458885c3b1f50629e46e74b5717c9d102acfaa8feec1e2c445277abb4bd54
  Stored in directory: c:\users\pc\appdata\local\pip\cache\wheels\ad\5c\ba\05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


In [22]:
metric_seqeval = load_metric("seqeval")
example = wnut["train"][2]

labels = [label_list[i] for i in example["ner_tags"]]
metric_seqeval.compute(predictions=[labels], references=[labels])

{'corporation': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0, 'number': 1},
 'overall_precision': 1.0,
 'overall_recall': 1.0,
 'overall_f1': 1.0,
 'overall_accuracy': 1.0}

In [23]:
def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    # Remove ignored index (special tokens)
    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric_seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [24]:
from transformers import EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir='./log_results',
    num_train_epochs=5,
    learning_rate=2e-5,
    per_device_train_batch_size=16,   
    per_device_eval_batch_size=64,
    weight_decay=0.01,
    warmup_steps=500, 
    eval_steps=60,
    save_steps=60,
    evaluation_strategy="steps",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_wnut["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience = 6)]
)

trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
C:\ProgramData\Anaconda3\envs\stance\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 4403
  Num Epochs = 5
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1380


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
60,No log,1.857654,0.003940,0.009268,0.005529,0.800222
120,No log,0.462477,0.000000,0.000000,0.000000,0.925612
180,No log,0.358537,0.000000,0.000000,0.000000,0.925612
240,No log,0.320890,0.315985,0.078777,0.126113,0.930999
300,No log,0.309467,0.475162,0.203892,0.285344,0.936343
360,No log,0.264187,0.447059,0.281742,0.345651,0.939421
420,No log,0.253361,0.556686,0.354958,0.433503,0.941943
480,No log,0.239884,0.508813,0.401297,0.448705,0.944765
540,0.536100,0.246245,0.553163,0.380908,0.451153,0.945663
600,0.536100,0.260673,0.528590,0.385542,0.445874,0.946689


The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1287
  Batch size = 64
Saving model checkpoint to ./log_results\checkpoint-60
Configuration saved in ./log_results\checkpoint-60\config.json
Model weights saved in ./log_results\checkpoint-60\pytorch_model.bin
tokenizer config file saved in ./log_results\checkpoint-60\tokenizer_config.json
Special tokens file saved in ./log_results\checkpoint-60\special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Runnin

TrainOutput(global_step=1020, training_loss=0.3020708860135546, metrics={'train_runtime': 2986.1804, 'train_samples_per_second': 7.372, 'train_steps_per_second': 0.462, 'total_flos': 432526398214206.0, 'train_loss': 0.3020708860135546, 'epoch': 3.7})

In [25]:
predictions, labels, _ = trainer.predict(tokenized_wnut["test"])
predictions = np.argmax(predictions, axis=2)

# Remove ignored index (special tokens)
true_predictions = [
    [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]
true_labels = [
    [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
    for prediction, label in zip(predictions, labels)
]

results = metric_seqeval.compute(predictions=true_predictions, references=true_labels)
results

The following columns in the test set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1287
  Batch size = 64


{'corporation': {'precision': 0.2597402597402597,
  'recall': 0.30303030303030304,
  'f1': 0.2797202797202797,
  'number': 66},
 'creative-work': {'precision': 0.35714285714285715,
  'recall': 0.176056338028169,
  'f1': 0.23584905660377353,
  'number': 142},
 'group': {'precision': 0.45,
  'recall': 0.10909090909090909,
  'f1': 0.17560975609756097,
  'number': 165},
 'location': {'precision': 0.5176470588235295,
  'recall': 0.5866666666666667,
  'f1': 0.5499999999999999,
  'number': 150},
 'person': {'precision': 0.623015873015873,
  'recall': 0.7319347319347319,
  'f1': 0.6730975348338694,
  'number': 429},
 'product': {'precision': 0.19230769230769232,
  'recall': 0.07874015748031496,
  'f1': 0.111731843575419,
  'number': 127},
 'overall_precision': 0.52026286966046,
 'overall_recall': 0.4402224281742354,
 'overall_f1': 0.47690763052208834,
 'overall_accuracy': 0.9478431875507674}

In [26]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertForTokenClassification.forward` and have been ignored: ner_tags, id, tokens. If ner_tags, id, tokens are not expected by `BertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1287
  Batch size = 64


{'eval_loss': 0.2339123785495758,
 'eval_precision': 0.52026286966046,
 'eval_recall': 0.4402224281742354,
 'eval_f1': 0.47690763052208834,
 'eval_accuracy': 0.9478431875507674,
 'eval_runtime': 68.067,
 'eval_samples_per_second': 18.908,
 'eval_steps_per_second': 0.309,
 'epoch': 3.7}

In [29]:
def tag_sentence(text:str):
    # convert our text to a  tokenized sequence
    #inputs = tokenizer(text, truncation=True, return_tensors="pt").to("cuda")
    inputs = tokenizer(text, truncation=True, return_tensors="pt")
    # get outputs
    outputs = model(**inputs)
    # convert to probabilities with softmax
    probs = outputs[0][0].softmax(1)
    # get the tags with the highest probability
    word_tags = [(tokenizer.decode(inputs['input_ids'][0][i].item()), id2tag[tagid.item()]) 
                  for i, tagid in enumerate (probs.argmax(axis=1))]

    return pd.DataFrame(word_tags, columns=['word', 'tag'])

In [30]:
text = """Celebrities and tourists from United States are 
flooding into Greece. But a harsh winter isn’t far off"""

print(tag_sentence(text))

           word         tag
0         [CLS]           O
1   celebrities           O
2           and           O
3      tourists           O
4          from           O
5        united  B-location
6        states  B-location
7           are           O
8      flooding           O
9          into           O
10       greece  B-location
11            .           O
12          but           O
13            a           O
14        harsh           O
15       winter           O
16          isn           O
17            ’           O
18            t           O
19          far           O
20          off           O
21        [SEP]           O


In [31]:
text="""Apple unveils all-new MacBook Air, 
supercharged by the new M2 chip"""

print(tag_sentence(text))

         word        tag
0       [CLS]          O
1       apple  B-product
2          un          O
3        ##ve          O
4       ##ils          O
5         all          O
6           -          O
7         new          O
8         mac  B-product
9      ##book  I-product
10        air  I-product
11          ,          O
12      super          O
13  ##charged          O
14         by          O
15        the          O
16        new          O
17         m2  B-product
18       chip          O
19      [SEP]          O


In [35]:
text="""FISH MEAL (UNFINISHED) (UNFIT FOR HUMAN CONSUMPTION) (SIP NO.13328/2021/DADF DTD 02.09.2021)"""
text="""Bu geziye kadar Bolonya ile ilişkimiz biraz tuhaftı ve bu noktada sanki kendisine bir özür borçluyuz gibi hissediyoruz. Özür dileriz Bolonya"""
print(tag_sentence(text))

      word tag
0    [CLS]   O
1       bu   O
2       ge   O
3     ##zi   O
4     ##ye   O
5       ka   O
6    ##dar   O
7       bo   O
8    ##lon   O
9     ##ya   O
10     ile   O
11      il   O
12   ##isk   O
13   ##imi   O
14     ##z   O
15      bi   O
16   ##raz   O
17      tu   O
18    ##ha   O
19    ##ft   O
20     ##ı   O
21      ve   O
22      bu   O
23      no   O
24    ##kt   O
25   ##ada   O
26    sank   O
27     ##i   O
28     ken   O
29   ##dis   O
30   ##ine   O
31      bi   O
32     ##r   O
33      oz   O
34    ##ur   O
35      bo   O
36    ##rc   O
37    ##lu   O
38    ##yu   O
39     ##z   O
40      gi   O
41    ##bi   O
42  hissed   O
43    ##iy   O
44    ##or   O
45    ##uz   O
46       .   O
47      oz   O
48    ##ur   O
49     dil   O
50   ##eri   O
51     ##z   O
52      bo   O
53   ##lon   O
54    ##ya   O
55   [SEP]   O


In [36]:
!rm -r log_results/

'rm' is not recognized as an internal or external command,
operable program or batch file.
